# enwiki-20210520-categorylinks Stats
- 23530 lines
- ~164,000,000 items
- 151,894,856 'page' items

```python
(10,'Redirects_from_moves','*..2NN:,@2.FBHRP:D6\x01\x17\x01\xdc\xbd\xdc\x0c','2014-10-26 04:50:23','','uca-default-u-kn','page')
```

In [7]:
import re
from typing import Iterator

import numpy as np

In [2]:
line_start = re.compile(b"INSERT INTO (.*?) VALUES ")
item_ex = re.compile(b"\((.*?),'(.*?)','(.*?)','(.*?)','(.*?)','(.*?)','(.*?)'\)")

interesting_category_exps = [
    re.compile(b"^FA-Class_(.*?)_articles"),
    re.compile(b"^FL-Class_(.*?)_articles"),
    re.compile(b"^A-Class_(.*?)_articles"),
    re.compile(b"^GA-Class_(.*?)_articles"),
    re.compile(b"^B-Class_(.*?)_articles"),
    re.compile(b"^C-Class_(.*?)_articles"),
]  # sorted by quality

interesting_category_lists = [
    [],  # FA
    [],  # FL
    [],  # A
    [],  # GA
    [],  # B
    [],  # C
]

In [3]:
class FileProcesser:
    def __init__(self):
        self.num_items = 0

    @staticmethod 
    def get_line_iterator(line) -> Iterator:
        if line_start.search(line) is not None:
            item_iter = item_ex.finditer(line)
            return item_iter
        else:
            return None


    def find_interesting_categories(self, line_iter):
        for entry in line_iter:
            if entry.group(7) != b"page":
                continue
            self.num_items += 1
            if self.num_items % 10000000 == 0:
                print(f"searched {self.num_items} items")
                for row in interesting_category_lists:
                    print(len(row))
            for i, exp in enumerate(interesting_category_exps):
                result = exp.match(entry.group(2))
                if result is not None:
                    interesting_category_lists[i].append([entry.group(1), entry.group(2), entry.group(7)])


    def process_line(self, line):
        line_iter = self.get_line_iterator(line)
        if line_iter is None:
            return 0
        self.find_interesting_categories(line_iter)

In [4]:
fp = FileProcesser()
with open("data/enwiki-20210520-categorylinks.sql", "rb") as file:
    for line in file:
        fp.process_line(line)

searched 10000000 items
8544
576
479
17227
67163
113283
searched 20000000 items
15286
2004
1533
40592
125190
263864
searched 30000000 items
20891
3205
2146
63108
179453
436431
searched 40000000 items
24469
4482
2782
79191
215932
555499
searched 50000000 items
26657
6339
3160
90337
239996
637791
searched 60000000 items
29285
8171
3582
101177
266839
717924
searched 70000000 items
30803
9598
3993
112608
289325
796615
searched 80000000 items
31680
10991
4350
121576
308619
864808
searched 90000000 items
32347
11787
4561
130394
325657
931752
searched 100000000 items
33476
12887
4675
138305
342344
999591
searched 110000000 items
34276
13659
4835
145303
356907
1063275
searched 120000000 items
34972
14305
4967
150585
372843
1130415
searched 130000000 items
35481
14821
5090
155764
388731
1203343
searched 140000000 items
36062
15124
5178
160299
404036
1272297
searched 150000000 items
36328
15447
5231
164692
419817
1363084


In [6]:
fp.num_items

151894856

In [5]:
for row in interesting_category_lists:
    print(len(row))

36335
15458
5239
164965
422165
1377104


In [15]:
interesting_category_lists[5][0]

[b'354', b'C-Class_Version_1.0_articles', b'page']

In [16]:
np_array = np.array(interesting_category_lists)

<ipython-input-16-f6a396bad7ea>:1: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  np_array = np.array(interesting_category_lists)


In [18]:
np.save("categorylinks", np_array)